In [1]:
import pandas as pd
import numpy as np
import tensorflow.keras.layers
from keras.src.layers import Lambda
from matplotlib import pyplot as plt
import tensorflow as tf
import math
import ydf  # Yggdrasil Decision Forests
from sklearn.model_selection import train_test_split
from wurlitzer import sys_pipes
import keras.layers as preprocessing
from sklearn.model_selection import KFold
import tqdm as tqdm
from sklearn.preprocessing import MinMaxScaler


%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

2025-03-29 08:13:19.456046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743250399.467694 2568358 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743250399.471190 2568358 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743250399.481043 2568358 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743250399.481052 2568358 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743250399.481053 2568358 computation_placer.cc:177] computation placer alr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid TensorFlow using all GPU memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        # Set TensorFlow to use only the first GPU (if multiple GPUs available)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
def train_and_eval(model, train_ds, test_ds = None):
    # Optionally, add evaluation metrics.
    model.compile(metrics=["mse"])
    rmse = 0

    with sys_pipes():
        model.fit(x=train_ds)

    if test_ds is not None:
        evaluation = model.evaluate(x=test_ds, return_dict=True)
        rmse = math.sqrt(evaluation["mse"])

    return rmse

def latlon_to_xyz(lat, lon):
    lat, lon = np.radians(lat), np.radians(lon)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return x, y, z

# Example: Normalize Cartesian coordinates between 0 and 1
def normalize_xyz(x, y, z):
    # Normalizing each coordinate to the [0, 1] range
    return (x + 1) / 2, (y + 1) / 2, (z + 1) / 2

In [4]:
ROOT_DIR = "temporary"
train_df = pd.read_csv(f'{ROOT_DIR}/Train.csv')
test_df = pd.read_csv(f'{ROOT_DIR}/Test.csv')
vocab_df = pd.read_csv(f'{ROOT_DIR}/variable_descriptions.csv')
admin_df = pd.read_csv(f'{ROOT_DIR}/zaf_adminboundaries_tabulardata.csv', sep=";")

admin_df = admin_df[["ADM4_PCODE", "AREA_SQKM", "ADM2_ID"]] # ADM3_ID
admin_df["AREA_SQKM"] = admin_df["AREA_SQKM"].str.replace(",", ".").astype(float)
train_df = pd.merge(train_df, admin_df, on="ADM4_PCODE", how="left")
test_df = pd.merge(test_df, admin_df, on="ADM4_PCODE", how="left")
label_column = "target"

default_columns = ["ward", "ADM4_PCODE"]
nul_cols = ["dw_12", "dw_13", "lan_13", "pw_08", "pw_07"] # Columns with null values
cat_columns = ["ADM2_ID"] # Categorical columns
ft_columns = default_columns + cat_columns

In [9]:
from sklearn.preprocessing import OneHotEncoder

scaler = MinMaxScaler()
# categorical_encoder = OneHotEncoder(sparse_output=False)
def preprocess_df(input_df, is_train=False):
    drop_cols = []
    df = input_df.copy()
    df = df.drop(nul_cols, axis=1)

    ## Create a new feature
    df["phi"] = df["total_individuals"] / df["total_households"]
    df["id_area"] = df["total_individuals"] / df["AREA_SQKM"]
    df["hs_area"] = df["total_households"] / df["AREA_SQKM"]

    ## Normalize some columns
    norm_cols = ["phi", "NL", "id_area", "hs_area"]
    # norm_cols = ["NL"]
    df[norm_cols] = scaler.fit_transform(df[norm_cols]) if is_train else scaler.transform(df[norm_cols])

    ## Encode categorical columns
    # encoded = categorical_encoder.fit_transform(df[cat_columns]) if is_train else categorical_encoder.transform(df[cat_columns])
    # encoded_df = pd.DataFrame(encoded, columns=categorical_encoder.get_feature_names_out(cat_columns))
    # df = pd.concat([df, encoded_df], axis=1)
    drop_cols = drop_cols + cat_columns
    
    # Transform lat and lon to xyz
    df["x"], df["y"], df["z"] = zip(*df.apply(lambda x: latlon_to_xyz(x["lat"], x["lon"]), axis=1))
    df["x"], df["y"], df["z"] = zip(*df.apply(lambda x: normalize_xyz(x["x"], x["y"], x["z"]), axis=1))

    drop_cols = drop_cols + default_columns
    if len(drop_cols) > 0:
        df.drop(drop_cols, axis=1, inplace=True)
    
    return df

## Test preprocess_df
# preprocess_df(train_ds_pd, is_train=True).head()

In [7]:
from sklearn.model_selection import GroupKFold
X = train_df.drop(label_column, axis=1)
y = train_df[label_column]
g = train_df["ADM2_ID"]

cv_rmse = []
test_rmse = []
for train_index, test_index in GroupKFold(n_splits=10, shuffle=False).split(X, y, g):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

    X_train = preprocess_df(X_train, is_train=True)
    X_val = preprocess_df(X_val, is_train=False)
    X_train["target"] = y_train
    X_val["target"] = y_val

    learner = ydf.RandomForestLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=32)
    evaluation = learner.cross_validation(X_train, folds=5)
    cv_rmse.append(evaluation.rmse)
    evaluation = learner.train(X_train).evaluate(X_val)
    test_rmse.append(evaluation.rmse)

np.mean(cv_rmse), np.mean(test_rmse), np.std(cv_rmse), np.std(test_rmse)

Train model on 2548 examples
Model trained in 0:00:00.277396
Train model on 2561 examples
Model trained in 0:00:00.282709
Train model on 2562 examples
Model trained in 0:00:00.283482
Train model on 2526 examples
Model trained in 0:00:00.285608
Train model on 2541 examples
Model trained in 0:00:00.288202
Train model on 2532 examples
Model trained in 0:00:00.281403
Train model on 2535 examples
Model trained in 0:00:00.282745
Train model on 2528 examples
Model trained in 0:00:00.273338
Train model on 2521 examples
Model trained in 0:00:00.280801
Train model on 2544 examples
Model trained in 0:00:00.289750


(3.523075212059068,
 3.863823675978626,
 0.061512679308861846,
 0.6737346758439062)

I think that the cv inside YDF use shuffle=True

In [8]:
all_models = {
    "GBT": ydf.GradientBoostedTreesLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=32),
    "RF": ydf.RandomForestLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=32),
}

for mn, md in all_models.items():
    cv_rmse = []
    test_rmse = []
    for train_index, test_index in GroupKFold(n_splits=10, shuffle=False).split(X, y, g):
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]

        X_train = preprocess_df(X_train, is_train=True)
        X_val = preprocess_df(X_val, is_train=False)
        X_train["target"] = y_train
        X_val["target"] = y_val

        evaluation = md.cross_validation(X_train, folds=5)
        cv_rmse.append(evaluation.rmse)
        evaluation = md.train(X_train).evaluate(X_val)
        test_rmse.append(evaluation.rmse)

    print(f"{mn}: {np.mean(cv_rmse):.2f} {np.mean(test_rmse):.2f} {np.std(cv_rmse):.2f} {np.std(test_rmse):.2f}")

Train model on 2548 examples
Model trained in 0:00:00.841200
Train model on 2561 examples
Model trained in 0:00:01.292755
Train model on 2562 examples
Model trained in 0:00:00.835092
Train model on 2526 examples
Model trained in 0:00:01.147864
Train model on 2541 examples
Model trained in 0:00:00.909121
Train model on 2532 examples
Model trained in 0:00:01.348094
Train model on 2535 examples
Model trained in 0:00:01.026823
Train model on 2528 examples
Model trained in 0:00:00.969327
Train model on 2521 examples
Model trained in 0:00:01.380517
Train model on 2544 examples
Model trained in 0:00:00.352337
GBT: 3.37 3.90 0.06 0.66
Train model on 2548 examples
Model trained in 0:00:00.287515
Train model on 2561 examples
Model trained in 0:00:00.282815
Train model on 2562 examples
Model trained in 0:00:00.283240
Train model on 2526 examples
Model trained in 0:00:00.270677
Train model on 2541 examples
Model trained in 0:00:00.280512
Train model on 2532 examples
Model trained in 0:00:00.278196

I think that splitting the data with GroupKFold is better because it trains the model with data from one zone and evaluates it with data from another zone. And we can remark that the RF model is better than the GBT mode using this approach.

I use the default features, excluding the columns with null values and the categorical columns. I also normalize some columns.

In [10]:
all_models = {
    "GBT": ydf.GradientBoostedTreesLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=32),
    "RF": ydf.RandomForestLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=32),
}

for mn, md in all_models.items():
    cv_rmse = []
    test_rmse = []
    for train_index, test_index in GroupKFold(n_splits=10, shuffle=False).split(X, y, g):
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]

        X_train = preprocess_df(X_train, is_train=True)
        X_val = preprocess_df(X_val, is_train=False)
        X_train["target"] = y_train
        X_val["target"] = y_val

        evaluation = md.cross_validation(X_train, folds=5)
        cv_rmse.append(evaluation.rmse)
        evaluation = md.train(X_train).evaluate(X_val)
        test_rmse.append(evaluation.rmse)

    print(f"{mn}: {np.mean(cv_rmse):.2f} {np.mean(test_rmse):.2f} {np.std(cv_rmse):.2f} {np.std(test_rmse):.2f}")

Train model on 2548 examples
Model trained in 0:00:01.470872
Train model on 2561 examples
Model trained in 0:00:00.602826
Train model on 2562 examples
Model trained in 0:00:01.533941
Train model on 2526 examples
Model trained in 0:00:01.166658
Train model on 2541 examples
Model trained in 0:00:01.453185
Train model on 2532 examples
Model trained in 0:00:01.415135
Train model on 2535 examples
Model trained in 0:00:01.475323
Train model on 2528 examples
Model trained in 0:00:01.169507
Train model on 2521 examples
Model trained in 0:00:01.066354
Train model on 2544 examples
Model trained in 0:00:01.386279
GBT: 3.29 3.81 0.07 0.56
Train model on 2548 examples
Model trained in 0:00:00.315993
Train model on 2561 examples
Model trained in 0:00:00.294629
Train model on 2562 examples
Model trained in 0:00:00.289265
Train model on 2526 examples
Model trained in 0:00:00.318273
Train model on 2541 examples
Model trained in 0:00:00.308953
Train model on 2532 examples
Model trained in 0:00:00.299581

I add new features to the model. I create a new feature phi, which is the ratio between the total number of individuals and the total number of households. I also create two new features, id_area and hs_area, which are the ratio between the total number of individuals and households and the area of the zone. I normalize these new features. And I also normalize the NL feature. And then I train the model. The best model is GBT with a RMSE of 3.81.

In [28]:
all_models = {
    "GBT": ydf.GradientBoostedTreesLearner,
    "RF": ydf.RandomForestLearner,
}

## Split data on train and test based on the zone
X = train_df.drop(label_column, axis=1)
y = train_df[label_column]
g = train_df["ADM2_ID"]

## Sample a subset of the zone
test_g = g.sample(n=6, random_state=42)
train_g = g[~g.isin(test_g)]
X_train, X_val = X[g.isin(train_g)], X[g.isin(test_g)]
y_train, y_val = y[g.isin(train_g)], y[g.isin(test_g)]

X_train = preprocess_df(X_train, is_train=True)
X_val = preprocess_df(X_val, is_train=False)
X_train["target"] = y_train
X_val["target"] = y_val


for mn, md in all_models.items():
    tuner = ydf.RandomSearchTuner(num_trials=20, automatic_search_space=True)
    model = md(label=label_column, task=ydf.Task.REGRESSION, num_threads=32, tuner=tuner)
    trainer = model.train(ds=X_train, valid=X_val if mn == "GBT" else None)
    evaluation = trainer.evaluate(X_val)
    outputs[mn] = {
        "evaluation": evaluation,
        "hp": model,
        "tuner": tuner,
    }

    print(f"{mn}: {evaluation.rmse:.2f}")

Train model on 2509 training examples and 313 validation examples
Model trained in 0:18:43.727930
GBT: 3.02
Train model on 2509 examples
Model trained in 1:07:58.387279
RF: 3.18


In [38]:
best_model_name = "GBT"
best_hp = outputs[best_model_name]["hp"].hyperparameters
best_model = all_models[best_model_name](label=label_column, task=ydf.Task.REGRESSION, num_threads=32, **best_hp)
trainer = best_model.train(ds=X_train)
evaluation = trainer.evaluate(X_val)


Train model on 2509 examples
Model trained in 0:00:01.414367


In [42]:
len(X_train.columns), len(X_val.columns)

(63, 63)

In [41]:
len(trainer.input_features()), len(trainer.input_feature_names())

(62, 62)

In [43]:
[col for col in X_train.columns if col not in trainer.input_feature_names()]

['target']

In [31]:
trainer.describe()

In [45]:
# List the available variable importances.
print(trainer.variable_importances().keys())

dict_keys(['NUM_NODES', 'SUM_SCORE', 'INV_MEAN_MIN_DEPTH', 'NUM_AS_ROOT'])


The best variable importance (VI) metric to use depends on your goal and the specific properties of your dataset and model. Here's how you can decide:

### **1. If you want a model-agnostic approach (more generalizable)**
   - **MEAN_{INCREASE, DECREASE}_IN_{metric}**
     - **Pros:** Measures the direct impact of a feature on performance (e.g., accuracy, AUC).
     - **Cons:** Can be computationally expensive, especially with large datasets.
     - **Best for:** Understanding the real-world impact of a feature on model performance.

### **2. If you're using Decision Forests (more interpretable)**
   - **SUM_SCORE**
     - **Pros:** Reflects how often a feature contributes to splits across trees.
     - **Cons:** Doesn't measure the direct impact on performance.
     - **Best for:** Identifying frequently used features in splits.

   - **NUM_AS_ROOT**
     - **Pros:** Highlights features that are often chosen for root splits (high impact early on).
     - **Cons:** May favor categorical features with high cardinality.
     - **Best for:** Finding highly influential features at the start of decision trees.

   - **NUM_NODES**
     - **Pros:** Measures overall usage of a feature in decision paths.
     - **Cons:** Doesn't necessarily correlate with importance for prediction accuracy.
     - **Best for:** Identifying frequently used features.

   - **INV_MEAN_MIN_DEPTH**
     - **Pros:** Prioritizes features that appear early in tree paths.
     - **Cons:** Might be biased toward features that correlate with others.
     - **Best for:** Understanding hierarchical importance in tree-based models.

### **Which one to use?**
- **For feature selection:** Start with **MEAN_DECREASE_IN_ACCURACY** or **MEAN_DECREASE_IN_AUC** (if AUC matters).
- **For Decision Forests:** Use **INV_MEAN_MIN_DEPTH** and **NUM_AS_ROOT** to identify the most crucial features.
- **For interpretability:** Combine multiple metrics, prioritizing **SUM_SCORE** and **NUM_AS_ROOT**.

We can see that the trainer doesn't compute the most important feature key by default. We skip this part.